## Load data

In [55]:
import theano
theano.config.mode = 'Mode'#theano.compile. (nan_is_error=True, inf_is_error=False, big_is_error=False)
theano.config.NanGuardMode.nan_is_error = True
theano.config.NanGuardMode.inf_is_error = False
theano.config.NanGuardMode.big_is_error = False
import numpy as np
from keras.utils.np_utils import to_categorical
import keras
import os
#os.putenv('PYTHONHASHSEED', '0')

In [16]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
datapath = '/media/sf_vmshared/PAMAP2_Dataset/slidingwindow512cleaned/'

In [4]:
ext = '.npy'
X_train = np.load(datapath+'X_train'+ext)
y_train_binary = np.load(datapath+'y_train_binary'+ext)
X_val = np.load(datapath+'X_val'+ext)
y_val_binary = np.load(datapath+'y_val_binary'+ext)
X_test = np.load(datapath+'X_test'+ext)
y_test_binary = np.load(datapath+'y_test_binary'+ext)

In [5]:
num_classes = y_train_binary.shape[1]
print(num_classes)

12


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam
import keras.backend as K

In [7]:
import sys
import os
import numpy as np
sys.path.insert(0, os.path.abspath('..'))

from mcfly import modelgen, find_architecture

## Replicate the NaN model

In [8]:
params = {'filters': np.array([98, 67]), 'fc_hidden_nodes': 62, 'regularization_rate': 0.03708604649827486, 'learning_rate': 0.00019850152863738226}

We make a copy of the model, to start training from fresh.

In [9]:
# Check some theano settings

print(K.floatx())
print(K.epsilon())
#K.set_epsilon(1e-5)
#K.set_floatx = 'float32'
print(theano.config.optimizer)
#theano.config.optimizer = 'fast_run'
print(theano.config.nvcc.fastmath)
print(theano.config.mode)

float32
1e-07
fast_run
False
NanGuardMode


In [10]:
theano.config.NanGuardMode.inf_is_error

False

We're going to train the model on the complete data set, and save the model after each epoch

In [18]:
np.random.seed(123)
model = modelgen.generate_CNN_model(X_train.shape, num_classes, 
                                              params['filters'], 
                                              params['fc_hidden_nodes'],
                                              params['learning_rate'], 
                                              params['regularization_rate'])

nr_epochs = 1
datasize = X_train.shape[0]#1000
history = model.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_val, y_val_binary), 
                              callbacks=[keras.callbacks.ModelCheckpoint(datapath+'weights2.{epoch:02d}-{val_loss:.2f}.hdf5')])

Train on 12497 samples, validate on 2007 samples
Epoch 1/1
12497/12497 [==============================] - 101s - loss: 5.9390 - acc: 0.8574 - val_loss: 0.2698 - val_acc: 0.9273


## Find NaN model

Although parameter set + seed above result in NaN loss for Dafne's computer, it does not for Chris'. The code below looks for convenient setups that result in NaN values.

In [59]:
def train_and_check(seed, dataset):
    np.random.seed(seed)
    model = modelgen.generate_CNN_model(dataset.shape, num_classes, 
                                                  params['filters'], 
                                                  params['fc_hidden_nodes'],
                                                  params['learning_rate'], 
                                                  params['regularization_rate'])

    nr_epochs = 1
    datasize = dataset.shape[0]#1000
    history = model.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
                  nb_epoch=nr_epochs, validation_data=(X_val[:1], y_val_binary[:1]), 

                        callbacks=[keras.callbacks.ModelCheckpoint(datapath+'weights2.{epoch:02d}-{val_loss:.2f}.hdf5')])
    return math.isnan(history.history['loss'][0]) or math.isnan(history.history['val_loss'][0])

In [56]:
dataset = X_train[:1]
nan_seeds = []
for seed in range(49,53):
    if train_and_check(seed, dataset):
        nan_seeds.append(seed)
        print('Found Nan with seed ' + str(seed))
print(nan_seeds)

Train on 1 samples, validate on 2007 samples
Epoch 1/1
1/1 [==============================] - 4s - loss: 10.9805 - acc: 0.0000e+00 - val_loss: 12.9344 - val_acc: 0.1345
Train on 1 samples, validate on 2007 samples
Epoch 1/1
1/1 [==============================] - 4s - loss: 10.7914 - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.1151
Found Nan with seed 50
Train on 1 samples, validate on 2007 samples
Epoch 1/1
1/1 [==============================] - 4s - loss: 10.6052 - acc: 0.0000e+00 - val_loss: 13.0485 - val_acc: 0.1345
Train on 1 samples, validate on 2007 samples
Epoch 1/1
1/1 [==============================] - 4s - loss: 10.9311 - acc: 0.0000e+00 - val_loss: 12.8164 - val_acc: 0.1345
[50]


Some seeds that resulted in nan loss on Chris' computer were 43, 67, 116, 224, 257, 272, 394 when using a dataset of :100 of the complete X_train. With only :1 of X_train I get a NaN loss with seed 50. So below should give nan loss on the small dataset.

In [102]:
dataset = X_train[:1]
np.random.seed(50)
model = modelgen.generate_CNN_model(dataset.shape, num_classes, 
                                              params['filters'], 
                                              params['fc_hidden_nodes'],
                                              params['learning_rate'], 
                                              params['regularization_rate'])

nr_epochs = 1
datasize = dataset.shape[0]#1000
history = model.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_val[:1], y_val_binary[:1]), 

                    callbacks=[keras.callbacks.ModelCheckpoint(datapath+'weights2.{epoch:02d}-{val_loss:.2f}.hdf5')])

output  Elemwise{clip,no_inplace}.0
target  activation_1940_target
Train on 1 samples, validate on 1 samples
Epoch 1/1
1/1 [==============================] - 0s - loss: 10.7914 - acc: 0.0000e+00 - val_loss: nan - val_acc: 1.0000


In [91]:
dataset = X_train[:1]
np.random.seed(50)
model = modelgen.generate_CNN_model(dataset.shape, num_classes, 
                                              params['filters'], 
                                              params['fc_hidden_nodes'],
                                              params['learning_rate'], 
                                              params['regularization_rate'])

In [78]:
model.predict_classes(X_val[:1])

1/1 [==============================] - 0s


array([7])

In [99]:
l = model.layers[-2]
l.get_weights()[0]

array([[  5.24276048e-02,  -1.87787071e-01,  -1.46753630e-02,
          1.31986126e-01,  -2.68284306e-02,  -2.14224085e-01,
         -1.54472515e-01,  -5.69397509e-02,  -8.19158927e-02,
          1.65570810e-01,  -1.21166378e-01,  -2.12710425e-01],
       [ -1.77742243e-01,   3.18614505e-02,   4.87366393e-02,
          3.24277952e-02,  -1.33670375e-01,   1.48252890e-01,
          2.77611483e-02,   2.04569340e-01,  -7.10648894e-02,
         -2.59519601e-03,  -7.71293640e-02,  -1.79101005e-01],
       [  1.37388378e-01,  -1.46523207e-01,   1.64005868e-02,
         -1.59035102e-01,   1.77222699e-01,  -1.10589050e-01,
          1.39046267e-01,   1.22961700e-01,  -1.36654735e-01,
         -9.07251760e-02,   3.29503939e-02,   4.80490327e-02],
       [  1.73144758e-01,   1.27379701e-01,  -1.41657859e-01,
         -5.48100844e-02,  -1.71345383e-01,   5.62886335e-02,
         -7.46957511e-02,  -1.70681894e-01,   2.12964043e-02,
          9.49259326e-02,  -3.99484746e-02,   1.43435135e-01],
    

In [118]:
import keras.backend.theano_backend as th
target = numpy.zeros_like(layer_output)
print(target.shape)
print(y_val_binary[:1])
print(y_val_binary[:1].shape)
#target[0, 4] = 1
target = y_val_binary[:1]


out = th.categorical_crossentropy(layer_output, target)
type(out)

(1, 12)
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
(1, 12)
output  Elemwise{clip,no_inplace}.0
<class 'theano.tensor.var.TensorVariable'>
target  [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


theano.tensor.var.TensorVariable

In [100]:
get_dens_layer_output = keras.backend.function([model.layers[0].input, keras.backend.learning_phase()],
                                  [model.layers[-1].output])
layer_output = get_dens_layer_output([X_val[:1], 0])[0]

In [116]:
layer_output

array([[ 0.00859882,  0.06163753,  0.00589006,  0.17214319,  0.08953378,
         0.16991124,  0.07851812,  0.31809509,  0.02173777,  0.039786  ,
         0.01596426,  0.01818413]], dtype=float32)

In [85]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
batchnormalization_481 (BatchNormal(None, 512, 9)      18          batchnormalization_input_481[0][0
____________________________________________________________________________________________________
convolution1d_961 (Convolution1D)  (None, 512, 98)     2744        batchnormalization_481[0][0]     
____________________________________________________________________________________________________
activation_1921 (Activation)       (None, 512, 98)     0           convolution1d_961[0][0]          
____________________________________________________________________________________________________
convolution1d_962 (Convolution1D)  (None, 512, 67)     19765       activation_1921[0][0]            
___________________________________________________________________________________________

In [47]:
history.history['loss']

[5.9389754558092234]

If we plot the training process, we see that the model is probably already overfitting in the first epoch. 

In [ ]:
find_architecture.plotTrainingProcess(history)

In [ ]:
#Load model from the  just before we get the NaNs
#model_copy = keras.models.copy.copy(best_model_copy)
#model_copy.load_weights(datapath+'weights.{epoch:02d}-{val_loss:.2f}.hdf5'.format(epoch=3, val_loss=0.22))
#model_copy.compile(best_model_copy.optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.evaluate(X_train, y_train_binary)

## Inspect model predictions

In [ ]:
datasize = X_val.shape[0]
probs = model.predict_proba(X_val[:datasize,:,:])
probs

In [ ]:
len(model.layers)

In [ ]:
model.layers[0].get_weights()

In [ ]:
# with a Sequential model
get_dens_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.layers[0].output])
layer_output = get_dens_layer_output([X_val, 0])[0]
layer_output

In [ ]:
score_val